In [3]:
import cv2
import numpy as np
import json
import os
from typing import Tuple, Optional

# --- CONFIGURATION ---
IP_ADDRESS = "10.151.72.66"
PORT = "8080"
URL = f"http://{IP_ADDRESS}:{PORT}/video"

# ALPHA_VALUE controls how the image is resized/cropped:
# 1.0 = All pixels are retained (you will see curved black borders).
# 0.0 = No black borders (the image is zoomed/cropped to valid pixels only).
ALPHA_VALUE = 0

def load_calibration_parameters(filepath: str) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
    """
    Loads intrinsic matrix and distortion coefficients from a JSON file.
    Note: Extrinsic parameters (rvecs, tvecs) are NOT needed for lens correction.
    """
    if not os.path.exists(filepath):
        print(f"ERROR: File '{filepath}' not found.")
        return None, None
        
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
            
        # Convert lists to NumPy arrays (Crucial for OpenCV)
        mtx = np.array(data["intrinsics"])
        dist = np.array(data["distortion"])
        
        print(f"Loaded calibration data from {filepath}")
        return mtx, dist
        
    except Exception as e:
        print(f"ERROR reading JSON: {e}")
        return None, None

def main():
    # 1. Load Calibration Data
    mtx, dist = load_calibration_parameters(CALIB_FILE)
    if mtx is None or dist is None:
        print("Exiting due to missing calibration data.")
        return

    # 2. Initialize Video Stream
    print(f"Connecting to {URL} ...")
    cap = cv2.VideoCapture(URL)

    if not cap.isOpened():
        print("ERROR: Could not open video stream. Check IP and Wi-Fi connection.")
        return
    else:
        print("Video stream established successfully.")

    # 3. Read one frame to setup dimensions
    ret, frame = cap.read()
    if not ret:
        print("ERROR: Could not read the first frame.")
        return

    h, w = frame.shape[:2]
    print(f"Frame resolution: {w}x{h}")

    # 4. Compute Rectification Maps (Optimization)
    # We do this ONCE before the loop. Calculating this inside the loop causes lag.
    
    # Get the new optimal camera matrix based on the alpha value
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), ALPHA_VALUE, (w,h))
    
    # Generate the Look-Up Tables (maps) for the remapping process
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
    
    # Extract Region of Interest (ROI) coordinates for optional cropping
    x, y, w_roi, h_roi = roi

    print("Starting rectification loop. Press 'q' to exit.")

    # 5. Main Processing Loop
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Stream ended or connection lost.")
            break
        
        # Apply the undistortion using the pre-calculated maps (Fastest method)
        undistorted_frame = cv2.remap(frame, mapx, mapy, cv2.INTER_LINEAR)
        
        # Display results
        # cv2.imshow("Original", frame) # Optional: Show raw footage
        cv2.imshow("Undistorted (Corrected)", undistorted_frame)

        # Exit logic
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded calibration data from calibration_data.json
Connecting to http://10.151.72.66:8080/video ...
Video stream established successfully.
Frame resolution: 1920x1080
Starting rectification loop. Press 'q' to exit.
